In [6]:
import pandas as pd
import os
import itertools
import json
import numpy as np
from CorpusGenerator import CorpusGenerator
from Text_Processing import tokenizer_stemming
from Text_Processing import clean_text_similarity
import yaml
from gensim.models.phrases import Phrases, Phraser


# load configuration file
with open(os.path.join(".", 'config', 'config.yml'), 'r', encoding='utf-8') as conf_file:
    config = yaml.load(conf_file, Loader=yaml.SafeLoader)

# generate whole corpus for general representation construction
all_corpus = CorpusGenerator(country_list=["TW", "EU"],
                             task="SIM", data_location="default",
                             generate_only_for=None, filtered=None)
corpus_dataframe = all_corpus.read_from_location()
tokenized_cleaned_corpus = list(
    corpus_dataframe["Text"].apply(clean_text_similarity).apply(lambda x: x.lower()).apply(tokenizer_stemming))

# bigram, trigram for gensim model
phrases = Phrases(tokenized_cleaned_corpus,
                  min_count=config["Modelling"]["PHRASER"]["config"]["MIN_COUNT"],
                  progress_per=config["Modelling"]["PHRASER"]["config"]["PROGRESS_PER"])

bigram = Phraser(phrases)
bi_sentences = bigram[tokenized_cleaned_corpus]
trigram = Phrases(bi_sentences)
sentences = trigram[bi_sentences]


Including all data
Including all data


In [7]:
bi_gram = pd.DataFrame(columns=["bigram","score"])
for phrase, score in phrases.export_phrases(tokenized_cleaned_corpus):
    bi_gram = bi_gram.append({"bigram":phrase, "score":score},ignore_index=True)

In [8]:
bi_gram = bi_gram.drop_duplicates().reset_index(drop=True)

In [13]:
bi_gram.to_excel("bigram_PMI score.xlsx")

In [10]:
bi_gram["bigram"] = bi_gram["bigram"].apply(lambda x: x.decode('UTF-8'))

In [15]:
tri_gram = pd.DataFrame(columns=["trigram","score"])
for phrase, score in trigram.export_phrases(bi_sentences):
    tri_gram = tri_gram.append({"trigram":phrase, "score":score},ignore_index=True)

In [16]:
#tri_gram = tri_gram.loc[:,["trigram","score"]]

In [17]:
tri_gram["trigram"] = tri_gram["trigram"].apply(lambda x: x.decode('UTF-8'))

In [19]:
tri_gram.sort_values(by="score",ascending=False).drop_duplicates().head()

,trigram,score
16828,6a 6b,6164.034722
64198,alkylaryl sulphonate,6002.508876
658,wrist fingers,5943.890625
75214,communicate_united nations_secretariat,5763.772727
87960,signed dated,5448.566406


In [20]:
#tri_gram.sort_values(by="score",ascending=False).drop_duplicates().to_excel("trigram.xlsx")